In [89]:
# Import ThingsapikeySecret)
import time # time thread-timeout
import config # for apikeys
import tweepy # for crawling Twitter

In [92]:
# Authenticate Tweepy, a Python Library for crawling Twitter via the Twitter API
# Get Apikey from here: https://developer.twitter.com/en
auth = tweepy.AppAuthHandler(
    config.APIKEY,
    config.SECRET_APIKEY)
api = tweepy.API(auth)

In [48]:
# Get my ID
myName = 'tobiashoelzer'
myID = api.get_user(myName).id

myID

3490529422

In [82]:
# Get my follows (friends)
myFriends = api.friends_ids(myID)

myFriends

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [79]:
# Get 2. Grade Follows (Friends)
secondGradeFriends = {}
for f_id in myFriends: # F is for Friends who do stuff together. U is for You and me. N is for Anywhere and anytime at all. Down here in the deep blue sea!
    try:
        # Get friends of current friend (of mine) and stores them in secondGradeFriends Dictionary
        f_friends = api.friends_ids(f_id)
        secondGradeFriends[f_id] = f_friends
    except tweepy.RateLimitError:
        # Prevents crashing if the 300 API-Call Limit from Twitters API caused an exeption
        # Tries to continue in 15 Minutes again.
        print('Sleep for 15 Minutes to avoid RateLimitErrors')
        time.sleep(15 * 60)
        f_friends = api.friends_ids(f_id)
        secondGradeFriends[f_id] = f_friends

In [80]:
# Reform data to fit into an edge list

# Opens edge file with write access
f = open("my-edgy-friends.edges", "w")
f.write(','.join(['User ID', 'User ID', 'Number of overlapping friends (weight)']) + '\n')

# Array which stores already calculated combinations
matched = []

# I know, there is for sure a better way to do this, but its late and I want to go home. :)
# Double iteration of myFriends
for f_id_i in myFriends:
    f_friends_i = secondGradeFriends[f_id_i]

    for f_id_j in myFriends:
        # If i and j are same users or i and j in combination was already calculated continue with next one
        if f_id_j == f_id_i or f'{f_id_j}-{f_id_i}' in matched:
            continue

        f_friends_j = secondGradeFriends[f_id_j]

        # Number which counts the amount of overlapping friends
        same_friends = 0

        # Iterate through friends of i and j to count overlapping friends
        for f_f_id_i in f_friends_i:
            for f_f_id_j in f_friends_j:
                if f_f_id_i == f_f_id_j:
                    same_friends += 1
        
        # If overlapping friends exists write them to the edge list [id of i, id of h, number of overlapping friends]
        if same_friends > 0:
            f.write(','.join([str(f_id_i), str(f_id_j), str(same_friends)]) + '\n')
        
        # Store the combination, so it doesn't calculated twice
        matched.append(f'{f_id_i}-{f_id_j}')

# Close file for os-security
f.close()